In [1]:
import os
import sys
import pandas as pd

root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_path)

In [2]:
from core.backtesting.optimizer import StrategyOptimizer

optimizer = StrategyOptimizer(root_path=root_path, database_name="optimization_database")
trials = pd.concat([optimizer.get_study_trials_df(study_name) for study_name in optimizer.get_all_study_names()])

trials['from_timestamp'] = pd.to_datetime(trials['from_timestamp'])
trials['to_timestamp'] = pd.to_datetime(trials['to_timestamp'])
trials['days_diff'] = (trials['to_timestamp'] - trials['from_timestamp']).dt.total_seconds() / 86400
trials['executors_per_day'] = trials['total_executors'] / trials['days_diff']
trials['volume_per_day'] = trials['total_volume'] / trials['days_diff']

import ast
def str_to_dict(x):
    return ast.literal_eval(x)

trials['trading_pair'] = trials['config'].apply(lambda x: str_to_dict(x)['trading_pair'])

['macd_bb_v1_with_optimizer',
 'pmm_simple_with_optimizer',
 'smugplug_v1_with_optimizer']

In [ ]:

def get_best_trials(df):
    final_df = pd.DataFrame()

    for trading_pair in df['trading_pair'].unique():
        trading_pair_df = df[(df['trading_pair'] == trading_pair) & (df['executors'])]
        filtered = trading_pair_df[
            (trading_pair_df['executors_per_day'] >= trading_pair_df['executors_per_day'].median()) &
            (trading_pair_df['volume_per_day'] >= trading_pair_df['volume_per_day'].median()) &
            (trading_pair_df['sharpe_ratio'] >= trading_pair_df['sharpe_ratio'].median()) &
            (trading_pair_df['max_drawdown_pct'] <= trading_pair_df['max_drawdown_pct'].median()) 
        ]

        if filtered.empty: 
            continue

        normalized_columns = ['executors_per_day', 'volume_per_day', 'sharpe_ratio', 'max_drawdown_pct']
        for col in normalized_columns:
            filtered[col] = filtered[col] / filtered[col].max()

        filtered['score'] = (
            filtered['executors_per_day'] *
            filtered['volume_per_day'] *
            filtered['sharpe_ratio'] *
            (1 / filtered['max_drawdown_pct'])
        )
        filtered['score'] = filtered['score'] / filtered['score'].max()
        filtered = filtered.dropna().sort_values(by='score', ascending=False).head(2)

        final_df = pd.concat([final_df, filtered[['study_name', 'config', 'trading_pair', 'max_drawdown_pct', 'executors_per_day', 'sharpe_ratio', 'volume_per_day', 'score']]])
    return final_df

# Call the function and pass the dataframe
best_trials = get_best_trials(trials)
best_trials

In [ ]:
import json
from core.services.backend_api_client import BackendAPIClient

client = BackendAPIClient()

for index, row in best_trials.iterrows():
    bot_config = str_to_dict(row.config)
    await client.add_controller_config(json.loads(bot_config.json()))
    await client.deploy_script_with_controllers(
        bot_name="test_1",
        controller_configs=[bot_config.id + ".yml"],
        image_name="dardonacci/hummingbot:latest",
        max_global_drawdown=50,
        max_controller_drawdown=20,
    )